In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
from xgboost import XGBClassifier
import time
import pandas as pd
import cv2
import numpy as np

In [ ]:
!pip install catboost


In [ ]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [ ]:

train = pd.read_csv('fashion-mnist_train.csv')
test = pd.read_csv('fashion-mnist_test.csv')
train.isnull().any().sum()
test.isnull().any().sum()

0

In [ ]:
train.shape

(60000, 785)

In [ ]:
test.shape

(10000, 785)

In [ ]:
df_train = train.copy()
df_test = test.copy()

In [ ]:
X_train= df_train.drop(['label'],axis = 1)
X_label = df_train['label']
y_test = df_test.drop(['label'],axis = 1)
y_label = df_test['label']

X_train = X_train.astype('float32')
y_test = y_test.astype('float32')
X_train /= 255.0
y_test /=255.0

In [ ]:
# X_train과 X_label을 하나의 데이터 프레임으로 합침
df_train = pd.concat([X_train, X_label], axis=1)

# 데이터 프레임을 섞음
df_train = df_train.sample(frac=1, random_state=42)

# 섞인 데이터 프레임에서 훈련 데이터와 레이블을 다시 분리
X_train = df_train.drop(['label'], axis=1)
X_label = df_train['label']

In [ ]:
X_label = X_label.values # change to array for mAP

In [ ]:
y_label = y_label.values # change to array for mAP

In [ ]:
from sklearn.metrics import auc
from collections import Counter
def calculate_mAP(preds,label):
    ## mAP calculation
    AP = []
    num_class = 10
    predict_label_count_dict = Counter(preds)
    predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

    # For each class
    for c, freq in predict_label_count_dict.items() :
        TP = 0
        FN = 0

        temp_precision = []
        temp_recall = []

        for i in range(len(preds)):
            # Calculate TP and FN
            if label[i] == c and preds[i] == c :
                TP += 1
            elif label[i] != c and preds[i] == c :
                FN += 1

            # Calculate precision and recall
            if TP+FN != 0:
                temp_precision.append(TP/(TP+FN))
                temp_recall.append(TP/freq)

        # Save the AP value of each class to AP array
        AP.append(auc(temp_recall, temp_precision))

    # Calculate mAP
    mAP = sum(AP) / num_class

    return mAP


In [ ]:
pca = PCA(n_components=400)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
y_test_pca = pca.transform(y_test)

X_train_PCA1 = pd.DataFrame(X_train_pca)
X_test_PCA1 = pd.DataFrame(y_test_pca)

In [ ]:
from catboost import CatBoostClassifier

# CatBoost 모델을 생성합니다.
catboost = CatBoostClassifier(iterations=1000, learning_rate=0.05, depth=6)

# 모델을 학습합니다.
catboost.fit(X_train_pca, X_label)

# 학습한 모델로 예측을 수행합니다.
catboost_train = catboost.predict(X_train_pca)
catboost_pred = catboost.predict(y_test_pca)

0:	learn: 2.1282690	total: 2.04s	remaining: 34m 1s
1:	learn: 1.9876700	total: 6.03s	remaining: 50m 10s
2:	learn: 1.8769624	total: 8.31s	remaining: 46m 1s
3:	learn: 1.7872558	total: 10.3s	remaining: 42m 51s
4:	learn: 1.7081374	total: 12.4s	remaining: 41m 14s
5:	learn: 1.6361987	total: 14.5s	remaining: 40m 4s
6:	learn: 1.5746224	total: 17s	remaining: 40m 9s
7:	learn: 1.5197798	total: 19.8s	remaining: 40m 50s
8:	learn: 1.4686239	total: 22.8s	remaining: 41m 50s
9:	learn: 1.4239278	total: 24.9s	remaining: 41m 1s
10:	learn: 1.3830268	total: 26.9s	remaining: 40m 20s
11:	learn: 1.3488011	total: 28.9s	remaining: 39m 40s
12:	learn: 1.3110616	total: 32.1s	remaining: 40m 35s
13:	learn: 1.2774911	total: 34.2s	remaining: 40m 6s
14:	learn: 1.2451196	total: 36.2s	remaining: 39m 39s
15:	learn: 1.2172008	total: 38.2s	remaining: 39m 12s
16:	learn: 1.1874703	total: 40.3s	remaining: 38m 51s
17:	learn: 1.1626954	total: 42.4s	remaining: 38m 33s
18:	learn: 1.1367502	total: 45.6s	remaining: 39m 12s
19:	learn: 

In [ ]:
from sklearn.metrics import classification_report
print("----CatBoost----")
catboost_train_score = accuracy_score(X_label, catboost_train)
catboost_pred_score = accuracy_score(y_label, catboost_pred)
print("Train Accuracy score: {}".format(catboost_train_score))
print("Test Accuracy score: {}".format(catboost_pred_score))
print(classification_report(y_label, catboost_pred))

----CatBoost----
Train Accuracy score: 0.9270333333333334
Test Accuracy score: 0.8762
              precision    recall  f1-score   support

           0       0.81      0.84      0.83      1000
           1       0.98      0.97      0.97      1000
           2       0.81      0.79      0.80      1000
           3       0.87      0.91      0.89      1000
           4       0.82      0.85      0.83      1000
           5       0.95      0.93      0.94      1000
           6       0.73      0.65      0.68      1000
           7       0.91      0.91      0.91      1000
           8       0.95      0.96      0.95      1000
           9       0.92      0.96      0.94      1000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [ ]:
import pickle
with open('catboost', 'rb') as f:
    model_load = pickle.load(f)

{'nan_mode': 'Min', 'eval_metric': 'MultiClass', 'iterations': 1000, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'random_score_type': 'NormalWithModelSizeDecrease', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 3, 'random_strength': 1, 'rsm': 1, 'boost_from_average': False, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'use_best_model': False, 'class_names': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'random_seed': 0, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'bagging_temperature': 1, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function':

In [ ]:
catboost_train = catboost_train.flatten()
catboost_pred = catboost_pred.flatten()
map_train = calculate_mAP(catboost_train,X_label)
map_test = calculate_mAP(catboost_pred,y_label)
print("Train Map score: {}".format(map_train))
print("Test Map score: {}".format(map_test))

Train Map score: 0.8621095612289652
Test Map score: 0.7702275315677474


In [ ]:
# model 저장
import pickle
with open('catboost', 'wb') as f:
    pickle.dump(catboost, f)